In [1]:
import pandas as pd
import requests
from plyer import notification
import datetime

In [2]:
#Criar a função de alerta de erro
def alerta():
    data_atual = datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")

    notification.notify(
        title= f"Erro {response.status_code}",
        message="Ocorreu uma falha no carregamentos dos dados da API" + "\nData atual: " + data_atual,
        app_name="Alerta",
        timeout=10)

In [3]:
#Consultar a API e gerar um dataframe com os feriados nacionais
ano = 2023
url =f"https://brasilapi.com.br/api/feriados/v1/{ano}"
response = requests.get(url)
if response.status_code == 200:
    data_json = response.json()
    feriados_nacionais = pd.DataFrame(data_json)
  
else:
    alerta()
#Alterar os nomes das colunas
feriados_nacionais = feriados_nacionais.rename(columns ={'date': 'Data', 'name': 'Feriado', 'type': 'Tipo'})
#Alterar o formato da data
feriados_nacionais['Data'] = pd.to_datetime(feriados_nacionais['Data']).dt.strftime('%d/%m/%Y')
#Deixar a coluna "Tipo" em português
feriados_nacionais['Tipo'] = feriados_nacionais['Tipo'].replace('national', 'Nacional')
#Consultar se existem valores nulos
feriados_nacionais.isna()
#Salvar em um arquivo csv
feriados_nacionais.to_csv('feriados_nacionais.csv', index=False)


In [4]:
# Consultar a API e gerar um dataframe com as corretoras ativas no Brasil
url2 ="https://brasilapi.com.br/api/cvm/corretoras/v1"
response = requests.get(url2, timeout= 15)
if response.status_code == 200:
    data_json2 = response.json()
    corretoras = pd.DataFrame(data_json2)
else:
    alerta()
#Limpar o dataframe deixando apenas as colunas com os dados de contato
corretoras_ativas = corretoras.loc[corretoras['status'] == 'EM FUNCIONAMENTO NORMAL']
dados = ['nome_social', 'email', 'municipio', 'telefone']
corretoras_ativas = corretoras_ativas[dados]
#Alterar nomes das colunas
corretoras_ativas = corretoras_ativas.rename(columns ={'nome_social': 'Nome', 'email': 'E-mail', 'municipio': 'Município', 'telefone': 'Telefone'})
#Formatar a coluna 'Telefone' no formato 'xxxx-xxxx' para melhor visualização
corretoras_ativas['Telefone'] = corretoras_ativas['Telefone'].astype(str).str[:4] + '-' + corretoras_ativas['Telefone'].astype(str).str[4:]
#Capitalizar a primeira letra de cada palavra na coluna 'Município'
corretoras_ativas['Município'] = corretoras_ativas['Município'].str.title()
#Consultar se existem valores nulos
corretoras_ativas.isna()
#Salvar em um arquivo csv
corretoras_ativas.to_csv('corretoras_ativas.csv', index=False)

In [5]:
#Consultar a API e gerar um dataframe com as taxas de juros vigentes no Brasil
url3 ="https://brasilapi.com.br/api/taxas/v1"
response = requests.get(url3)
if response.status_code == 200:
    data_json3 = response.json()
    taxas = pd.DataFrame(data_json3)
else:
    alerta()

#Alterar nomes das colunas
taxas = taxas.rename(columns= {'nome': 'Nome', 'valor': 'Valor (%)'})
#Consultar se existem valores nulos
taxas.isna()
#Salvar em um arquivo csv
taxas.to_csv('taxas.csv', index=False)